# InstaBot
Automation tasks for instagram to get more reactions or followers and scrapping people information.

Install web drivers to use the latest version of selenium.
https://selenium-python.readthedocs.io/

In [1]:
!pip install webdriver-manager

You should consider upgrading via the '/home/aingeru/anaconda3/bin/python -m pip install --upgrade pip' command.


Install instaloader, it provides helpful features related to instagram stuff.
https://instaloader.github.io/

In [2]:
!pip install instaloader

You should consider upgrading via the '/home/aingeru/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import random
import sys
import time
import os
import csv
import glob
import pandas as pd
import instaloader

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
from getpass import getpass
from datetime import datetime 

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


# download the latest chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - There is no [linux64] chromedriver for browser 90.0.4430 in cache
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
[WDM] - Driver has been saved in cache [/home/aingeru/.wdm/drivers/chromedriver/linux64/90.0.4430.24]


# Login 

In [5]:
def login(driver, username, password):
    driver.get("https://www.instagram.com/")
    time.sleep(2)
    # login_button = driver.find_element_by_xpath("//a[@href='/accounts/login/?source=auth_switcher']")
    # login_button.click()
    # time.sleep(2)
    user_name_elem = driver.find_element_by_xpath("//input[@name='username']")
    user_name_elem.clear()
    user_name_elem.send_keys(username)
    password_elem = driver.find_element_by_xpath("//input[@name='password']")
    password_elem.clear()
    password_elem.send_keys(password)
    password_elem.send_keys(Keys.RETURN)
    time.sleep(2)
    


In [6]:
# driver = webdriver.Chrome('./chromedriver')
USER = 'arcangel.zip'
PASSWORD = getpass()
login(driver, USER, PASSWORD)

········


# Like comments
Like all comments for a post. It may not like all comments if there are many of them due to instagram blocks some actions temporarily if we do it several times in a short time.

In [7]:
'''
Action blocked html 
<div class="pbNvD  fPMEg   " role="dialog">
    <div class="piCib">
        <div class="_08v79">
            <h3 class="_7UhW9     LjQVu     qyrsm KV-D4        uL8Hv         ">
                Action Blocked
            </h3>
            <div class="_7UhW9   xLCgt      MMzan   _0PwGv       uL8Hv         ">
                This action was blocked. Please try again later. We restrict certain content and actions to protect our community. Tell us if you think we made a mistake.</div></div><div class="mt3GC"><button class="aOOlW  bIiDR  " tabindex="0">Report a Problem</button><button class="aOOlW   HoLwm " tabindex="0">OK</button></div></div></div>
'''
def deal_blocked_action():
    div = driver.find_element_by_class_name('mt3GC')
    btn = div.find_elements_by_tag_name('button')
    button_ok = btn[1].click()
    
def __load_comments(n, delay=2):
    '''Click the watch more comments in the post n times'''
    print(f'There are {n} comments, loading them...')
    n = n // 12  # 12 comments updated at time
    for _ in tqdm(range(n), total=n, position=0, leave=True):  # force the tqdm bar to stay in the same line with position=0 and leave=True
        try:
            btn = driver.find_element_by_xpath("//button[@class='dCJp8 afkep']")
        except NoSuchElementException as e:
            break  # we have all the comments loaded
        btn.click()
        time.sleep(delay)


In [8]:
def __love_comment(comment):
    svgs = comment.find_elements_by_tag_name('svg')
    for svg in svgs:
        attr = svg.get_attribute('aria-label')
        if attr == 'Unlike':  # we've loved the button
            return True
        elif attr == 'Like':
            try:
                svg.click()
            except Exception as e:
                print("[!] IG got angry man, stop")  # love click limit reached
                return False
            return True
        
def __get_nick_from_comment(ul_comment):
    h3 = ul_comment.find_element_by_tag_name('h3')
    a = h3.find_element_by_tag_name('a')
    return a.text

In [ ]:
def __load_previous_nicknames_loved():
    if os.path.exists('loved.txt'):  # we have loved previous comments
        fname = 'loved.txt'
        with open(fname, 'r') as fp:
            loved = set(x for x in fp.read().split('\n'))
        loved.remove('') 
        print(f'{len(loved)} already loved')
        return loved
    return None

In [46]:
    
def like_comments(url, comments_count, loaded=False):
    if not loaded:
        driver.get(url)
        time.sleep(2)
        __load_comments(comments_count)
    print("[OK] All comments in the screen")
    loved_hist = __load_previous_nicknames_loved()
    comments = driver.find_elements_by_class_name('Mr508')
    ul_comments = filter(lambda c: c.tag_name == 'ul', comments)
    with open('loved.txt', 'a+') as fp:
        for comment in tqdm(ul_comments, total=comments_count, leave=True, position=0 ):
            nick = __get_nick_from_comment(comment)
            if loved_hist and nick in loved_hist:
                continue
            if not __love_comment(comment):
                return False  # IG is upset and thinks I'm a bot haha
            fp.write(nick + '\n')
            time.sleep(random.randint(1, 8))  # delay randomly to avoid anti-bot algorithm
    return True


In [52]:
# post_url = "https://www.instagram.com/p/B_ghUHuFtVr/"
# post_url = 'https://www.instagram.com/p/B_liTk6BHLe/'
post_url = 'https://www.instagram.com/p/CN3mHXjFatM/'
updated_comms = True
like_comments(post_url, 1235, updated_comms)

[OK] All comments in the screen
1162 already loved


 99%|█████████▉| 1220/1235 [01:42<00:01, 11.93it/s]


True

# Followers and following
Know who is following you and who you are following.

In [7]:
def get_inner_html(elem):
    return elem.get_attribute('innerHTML')

def click_post_followers_following(index):
    '''You shoud be in the target profile before execute the function.
    If index == 0 post else if index == 1 followers else if index == 2 following else error will be raised. '''
    ul_post_followers_following = driver.find_element_by_class_name("k9GMp ")
    li_elems = ul_post_followers_following.find_elements_by_tag_name('li')
    li_selected = li_elems[index]
    li_selected.find_element_by_tag_name('a').click() 
    
    # how many li are there
    return int(li_selected.find_element_by_tag_name('span').text)

def get_nicknames(people_count) -> set:
    '''We'll scrap nicknames by looking followers, following field in our account  
    '''    
    print('Scrolling...')
    div = driver.find_element_by_class_name('isgrP')
    ul_scroll = div.find_element_by_tag_name('ul')
    for _ in tqdm(range(people_count // 12)):  # Insta loads 12 elements per upload
        driver.execute_script('arguments[0].scrollIntoView(false)', ul_scroll)  # scroll all the way down to load all li elements inside ul list
        time.sleep(1)  # wait to load html elements
    
    li_people = driver.find_elements_by_class_name('wo9IH')  # it may find elements inside ul_scoll variable
    people_names = set()
    print('Appending nicknames in list...')
    try:
        for li_elem in tqdm(li_people, total=len(li_people), leave=True, position=0):
            div_elem = li_elem.find_element_by_class_name('d7ByH')
            a_elem = div_elem.find_element_by_tag_name('a')
            people_names.add(a_elem.text)
    except Exception as e:
        print(e)
    return people_names


In [8]:
def __get_people(option, url_profile):
    driver.get(url_profile)
    time.sleep(2)  # call time after update the document
    n = click_post_followers_following(option)
    print(f'Followers: {n}') if option == 1 else print(f'Following: {n}')    
    time.sleep(2)
    return get_nicknames(n)

def get_followers(username):
    url_profile = f'https://www.instagram.com/{username}/'
    return __get_people(1, url_profile)

def get_following(username):
    url_profile = f'https://www.instagram.com/{username}/'
    return __get_people(2, url_profile)

In [9]:
L = instaloader.Instaloader()
# Login or load session
L.login(USER, PASSWORD)        # (login)
# L.interactive_login(USER)      # (ask password on terminal)
# L.load_session_from_file(USER) # (load session created w/  #  `instaloader -l USERNAME`)

In [10]:
profile = instaloader.Profile.from_username(L.context, USER)  # Obtain profile metadata
followers = set(followee.username for followee in profile.get_followers())
following = set(followee.username for followee in profile.get_followees())

create a log directory to save people whom you had interacted with

In [11]:
now = datetime.now()
dir_name = str(now)
dir_log = './logs/' + dir_name
os.mkdir(dir_log)

In [12]:
print('Followers: ', len(followers), 'followee: ', len(following))
with open(dir_log + '/followers.txt', 'w') as fp:
    fp.write('\n'.join(followers))
with open(dir_log + '/following.txt', 'w') as fp:
    fp.write('\n'.join(following))
non_following_back = following - followers
with open(dir_log + '/non_following_back.txt', 'w') as fp:
    fp.write('\n'.join(non_following_back))
print('Non following back: ', len(non_following_back))

Followers:  866 followee:  915
Non following back:  215


load a whitelist if you don't mind the person is not following you

In [13]:
def get_whitelist():
    whitelist = set()
    file_list = glob.glob('./whitelist/*txt')
    for fpath in file_list:
        with open(fpath, 'r') as fp:
            whitelist = whitelist | set(fp.read().split('\n'))
    return whitelist - {''}

In [14]:
whitelist = get_whitelist()
removables = non_following_back - whitelist
len(removables)

122

## Remove non followers

In [15]:
def is_current_private():
    '''We assume we are following him. We click the unfollow button, then read if IG notifies us if it's private'''
    try:
        btn_following = driver.find_element_by_xpath("//span[@aria-label='Following']")
    except NoSuchElementException:
        print('You were not following him')
        return False
    btn_following.click()
    time.sleep(4)
    dialog = driver.find_element_by_xpath("//div[@role='dialog']")
    unfollow_btn, cancel_btn = dialog.find_elements_by_tag_name('button')
    msg = dialog.find_element_by_class_name('_08v79')
    private = 'If you change your mind' in msg.text  # if IG is in English it will work
    cancel_btn.click()
    time.sleep(4)
    return private

In [16]:

def get_user_info(username=None):
    '''Get basic info from users. Window must be maximized in order to work.'''
    # load user's profile
    if username is not None:
        username_url = f'https://www.instagram.com/{username}/'
        driver.get(username_url)
        time.sleep(random.randint(7, 15))

    # find ul elements (required for the count fields)
    ul = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul')
    li_list = ul.find_elements_by_tag_name('li')
    fetch_text = lambda e: e.find_element_by_class_name('g47SY ').text
    
    posts = fetch_text(li_list[0])
    followers = fetch_text(li_list[1])
    following = fetch_text(li_list[2])
    bio_elem =  driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[2]')
    return posts, followers, following, bio_elem.text

In [17]:
def unfollow_current(wait_dialog_secs=6):
    '''Unfollow the current profile where we are'''
    # unfollow him
    try:
        btn_following = driver.find_element_by_xpath("//span[@aria-label='Following']")
    except NoSuchElementException:
        print('You were not following him')
        return None
    btn_following.click()
    time.sleep(wait_dialog_secs)  # wait dialog
    dialog = driver.find_element_by_xpath("//div[@role='dialog']")
    unfollow_btn, cancel_btn = dialog.find_elements_by_tag_name('button')
    msg = dialog.find_element_by_class_name('_08v79').text
    unfollow_btn.click()
    return 'If you change your mind' in msg  # is private ?

In [18]:
df = pd.read_csv('./txt/unfollowed.csv')  # load dataframe of people you have unfollowed previously
removables = removables - set(df.nick)

In [19]:
fp = open('./txt/unfollowed.csv', mode='a+')
writer = csv.writer(fp)  # csv ready to append information

In [20]:
iterables = list(removables)
# batch_size = min(100, len(iterables))

In [21]:
bugs = []
for nick in tqdm(iterables, total=len(iterables), position=0, leave=True):
    # change to user profile
    username_url = f'https://www.instagram.com/{nick}/'
    driver.get(username_url)
    time.sleep(random.randint(20, 30))
    
    # verify if user exists
    try:
        h2 = driver.find_element_by_xpath("//h2[@class='VnYfv']")
        print(f"user {nick} not found")
        bugs.append(nick)
        continue
    except NoSuchElementException:
        pass
    
    posts, followers, following, bio = get_user_info()  # you also change to profile
    private = unfollow_current()
    if private is None:
        print(f"user {nick} hadn't been followed")
        bugs.append(nick)
    else:
        writer.writerow([nick, posts, followers, following, bio, private])
        fp.flush()  # important to reflect rows in csv file
    time.sleep(random.randint(5, 10))  # watch dialog who detect bots


100%|██████████| 114/114 [1:18:01<00:00, 41.07s/it]


In [48]:
fp.close()
removables = removables - set(bugs)
print(bugs)

## Follow accounts from files

In [18]:
def read_lines(fpath, ):
    with open(fpath, 'r') as fp:
        return fp.read().split('\n')

In [108]:
nicks = read_lines('./txt/2/nicks.txt')
header = ['nick', 'posts', 'followers', 'following', 'info', 'time_followed']
people_info, bugs = [], []
df = pd.read_csv("./txt/unfollowed.csv")

In [ ]:
print(len(nicks))
for followed in df.nick.values:
    if followed in nicks:
        print(followed)
        nicks.remove(followed)
print(len(nicks))

In [26]:
for nick in tqdm(nicks):
    follow_pressed = False
    try:
        t = get_user_info(nick)  #     return posts, followers, following, bio_elem.text
        btns = driver.find_elements_by_tag_name('button')
        for b in btns:
            if b.text == 'Follow':
                b.click()
                follow_pressed = True
                break
    except Exception as e:
        bugs.append((nick,) + (str(e),))
        print(e)
        continue
    if not follow_pressed:
        continue

    # format row
    row = (nick,) + t + (str(datetime.now()),)
    icon = driver.find_elements_by_class_name('smsjF')
    row = row + (1,) if icon else row + (0,)  # pubilc cell
    people_info.append(row)


 12%|█▏        | 29/238 [06:11<45:01, 12.93s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/header/section/ul"}
  (Session info: chrome=89.0.4389.90)



 21%|██        | 50/238 [10:43<40:44, 13.00s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/header/section/ul"}
  (Session info: chrome=89.0.4389.90)



 23%|██▎       | 55/238 [11:46<40:20, 13.23s/it]

...

 90%|█████████ | 215/238 [46:13<04:49, 12.60s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/header/section/ul"}
  (Session info: chrome=89.0.4389.90)



 93%|█████████▎| 221/238 [47:28<03:17, 11.61s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/header/section/ul"}
  (Session info: chrome=89.0.4389.90)



100%|██████████| 238/238 [50:46<00:00, 12.80s/it]


In [31]:
with open('./csv/followed.csv', mode='w') as fp:
    writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for t in people_info:
        writer.writerow(t)

with open('./csv/bug.csv', mode='w') as fp:
    writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for t in bugs:
        writer.writerow(t)        

## Active followers
People who have reacted to your posts.

In [54]:
PROFILE = USER
profile = instaloader.Profile.from_username(L.context, PROFILE)

likes = set()
print("Fetching likes of all posts of profile {}.".format(profile.username))
for post in profile.get_posts():
    print(post)
    likes = likes | set(post.get_likes())



Fetching likes of all posts of profile arcangel.zip.
<Post CNNrKSLLcyo>
<Post CMFjBKgH6ZX>
<Post CLFdxCCnJEQ>
<Post CJRe7QTnPZY>
<Post CFN1zphnen->
<Post B_Dl_4-J_Tk>
<Post B-DWtuYnSbl>
<Post B89r59anrkZ>
<Post B82HhRTgQbE>
<Post B8rY1Zmq1rh>
<Post B4k4Soig2Th>
<Post B37pBSRI6qH>
<Post B27RHEsoJ3N>
<Post B2MlGCoFmah>


In [55]:
likes

{<Profile 15_01_001 (3258544166)>,
 <Profile 3.s_s_s (29920713782)>,
 <Profile 3angels.3 (3645321338)>,
 <Profile __jowisz (3588727427)>,
 <Profile _andres_glz (607347757)>,
 <Profile _aris.angel_ (45802382444)>,
 <Profile _awen9_ (11403233059)>,
 <Profile _bautistafc25 (1432091754)>,
 <Profile _e.denisse (32950632)>,
 <Profile _eimlka_ (354021121)>,
 <Profile _howlettjames (435488756)>,
 <Profile _iampolaris_ (312064794)>,
 <Profile _jimenaperalta (3122905893)>,
 <Profile _ken_hye (3298152626)>,
 <Profile _maaira3 (3136685700)>,
 <Profile _meinthesea (36271893)>,
 <Profile _paupacheco (1267434741)>,
 <Profile _xmarinx (1246074007)>,
 <Profile _xresilienciax_ (5496037902)>,
 <Profile _yess.tart_ (3058912933)>,
 <Profile a.a_jatzibe (19080382499)>,
 <Profile a_vctrhg (1795012818)>,
 <Profile aamacalli_ (3163869120)>,
 <Profile aangelitatorres22 (3137800562)>,
 <Profile abapok (2246539234)>,
 <Profile abii_lv (4104843741)>,
 <Profile abz_fatemeh (15400315249)>,
 <Profile adriaan.lares (1

In [57]:
print("Storing active users into file.")
with open("active-users.txt", 'w') as f:
    for user in likes:
        print(user.username, file=f)

Storing active users into file.


In [56]:
print("Fetching followers of profile {}.".format(profile.username))
followers = set(profile.get_followers())

ghosts = followers - likes

print("Storing ghosts into file.")
with open("inactive-users.txt", 'w') as f:
    for ghost in ghosts:
        print(ghost.username, file=f)

Fetching followers of profile arcangel.zip.
Storing ghosts into file.


In [63]:
for u in likes:
    pass

In [73]:
posts = profile.get_posts()

In [77]:
for p in posts:
    break

In [80]:
p.date

datetime.datetime(2019, 9, 9, 16, 6, 33)

In [81]:
type(posts)

instaloader.nodeiterator.NodeIterator

In [83]:
now

datetime.datetime(2021, 4, 21, 21, 46, 44, 714024)